
# 🚨 Crisis Mode Risk Engine

_Date generated: 2025-09-03_

This notebook simulates a **Crisis Mode Risk Engine**, designed for hedge-fund style stress testing and emergency playbooks.

**Contents**
1. Synthetic portfolio & factor data (loads CSVs if available)
2. Crisis detection (volatility spikes, correlation breakdowns, liquidity shocks, drawdowns)
3. Crisis playbooks (de-risk, flight-to-quality, vol targeting)
4. Scenario shocks (1987 crash, GFC, COVID-19)
5. Backtest with & without crisis mode (equity, drawdowns, metrics)
6. Dashboard plots


## 0) Parameters

In [ ]:

PATH_RETURNS = "data/returns.csv"

N_ASSETS = 8
N_DAYS = 500

CRISIS_VOL_TH = 0.03   # 3% daily vol trigger
CRISIS_DD_TH  = -0.15  # -15% drawdown
CRISIS_CORR_TH = 0.8   # correlation breakdown if > 0.8


## 1) Setup

In [ ]:

import os, math, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
pd.options.display.float_format = "{:,.4f}".format
np.random.seed(42)


## 2) Load or Simulate Returns

In [ ]:

def load_returns(path=PATH_RETURNS, n_assets=N_ASSETS, n_days=N_DAYS):
    if os.path.exists(path):
        return pd.read_csv(path, parse_dates=["date"]).set_index("date")
    rng = np.random.default_rng(1)
    dates = pd.bdate_range("2023-01-01", periods=n_days)
    mu = rng.normal(0.0005, 0.0003, n_assets)
    sd = rng.uniform(0.01, 0.03, n_assets)
    data = [rng.normal(mu[i], sd[i], len(dates)) for i in range(n_assets)]
    return pd.DataFrame(np.array(data).T, index=dates, columns=[f"A{i}" for i in range(n_assets)])

rets = load_returns()
rets.head()


## 3) Crisis Detection

In [ ]:

def rolling_vol(r, win=20): return r.rolling(win).std() * np.sqrt(252)
def drawdown_curve(nav): return nav / nav.cummax() - 1.0

vol = rets.rolling(20).std().mean(axis=1)
eq = (1+rets.mean(axis=1)).cumprod()
dd = drawdown_curve(eq)
corr = rets.rolling(60).corr().groupby(level=0).mean().mean(axis=1)

crisis_flag = (vol > CRISIS_VOL_TH) | (dd < CRISIS_DD_TH) | (corr > CRISIS_CORR_TH)
crisis_flag = crisis_flag.astype(int)
crisis_flag.tail()


## 4) Crisis Playbooks

In [ ]:

def apply_playbook(rets, crisis_flag, mode="derisk"):
    w = np.ones(rets.shape[1]) / rets.shape[1]
    eq = [1.0]
    for t,(dt,row) in enumerate(rets.iterrows()):
        if crisis_flag.iloc[t]==1:
            if mode=="derisk":
                w = w*0.5
            elif mode=="quality":
                w = np.array([1 if i==0 else 0 for i in range(len(w))]) # assume A0 is safe asset
            elif mode=="volcap":
                if np.std(row)*np.sqrt(252) > CRISIS_VOL_TH:
                    w = w*0.7
        eq.append(eq[-1]*(1+np.dot(w,row.values)))
    return pd.Series(eq[1:], index=rets.index)

eq_base = (1+rets.mean(axis=1)).cumprod()
eq_derisk = apply_playbook(rets, crisis_flag, "derisk")
eq_quality = apply_playbook(rets, crisis_flag, "quality")
eq_volcap = apply_playbook(rets, crisis_flag, "volcap")


## 5) Equity Curves

In [ ]:

plt.figure(figsize=(10,4))
plt.plot(eq_base, label="Base")
plt.plot(eq_derisk, label="Crisis Derisk")
plt.plot(eq_quality, label="Flight to Quality")
plt.plot(eq_volcap, label="Vol Target")
plt.legend(); plt.title("Crisis Mode Playbooks"); plt.show()


## 6) Scenario Shocks

In [ ]:

def apply_shock(rets, shock=-0.2, day=200):
    rets2 = rets.copy()
    if day < len(rets2):
        rets2.iloc[day] += shock
    return rets2

rets_crash = apply_shock(rets, shock=-0.25, day=150)
eq_crash = (1+rets_crash.mean(axis=1)).cumprod()

plt.figure(figsize=(10,3))
plt.plot(eq_crash, label="Crash Shocked")
plt.title("Scenario Shock (1987 style)")
plt.show()


## 7) Metrics

In [ ]:

def sharpe(r): return r.mean()/r.std()*np.sqrt(252)
def maxdd(eq): return (eq/eq.cummax()-1).min()

metrics = pd.DataFrame({
    "Base": [sharpe(rets.mean(axis=1)), maxdd(eq_base)],
    "Derisk": [sharpe(eq_derisk.pct_change().dropna()), maxdd(eq_derisk)],
    "Quality": [sharpe(eq_quality.pct_change().dropna()), maxdd(eq_quality)],
    "VolCap": [sharpe(eq_volcap.pct_change().dropna()), maxdd(eq_volcap)]
}, index=["Sharpe","MaxDD"])
metrics
